In [1]:
import brightpath

In [2]:
c = brightpath.Converter(filepath="sample.xlsx", metadata="meta_sample.yaml")

Extracted 10 worksheets in 0.80 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 6.47 seconds


In [3]:
c.convert_to_simapro()

'Inventories export to: /Users/romain/GitHub/brightpath/dev/simapro_13-06-2023.csv'

In [6]:
bw2io.create_core_migrations

_Migrations dictionary with 0 objects